## test cells?


In [ ]:
var importer = require('../Core');
var path = require('path');
var Module = require('module').Module;
var Mocha = require('mocha');

// call test runner
var mocha;
if(typeof mocha === 'undefined') {
    mocha = new Mocha({
        ui: 'bdd',
        reporter: 'list',
        timeout: 10000,
    });
}

function resetTests(suite) {
    suite.tests.forEach(function(t) {
        delete t.state;
        t.timedOut = false;
    });
    suite.suites.forEach(resetTests);
    suite.tests = [];
}

function testCells(cells, ctx = {}) {
    resetTests(mocha.suite);
    if(!cells) {
        cells = 'test test runner';
    }
    if(!cells[0].code) {
         cells = typeof cells === 'string'
            && (!cells.includes('.ipynb') || cells.includes('['))
             ? [importer.interpret(cells)]
             : importer.interpret(cells);
    }
    cells.forEach(r => {
        mocha.suite.emit('pre-require', ctx, r.id, mocha);
        Object.assign(ctx, Mocha);
        try {
            var required = r.runInNewContext(ctx, {useCache: false});
            Object.assign(ctx, required);
        } catch (e) {
            console.log(e);
        }
        mocha.suite.emit('require', required, r.id, mocha);
        mocha.suite.emit('post-require', ctx, r.id, mocha);    
    });
    return new Promise(resolve => mocha.run(function (failures) {
        resolve(failures);
    }));
}
module.exports = testCells;


## test runner?

watch files run tests?

How to use: 

```
node -e "require('./Core').import('watch files run tests')('**/zuora*.ipynb', 'zuora to eloqua.ipynb')"

node -e "require('./Core').import('watch files run tests')('**/*.ipynb', 'test test runner')"```

TODO: restart this test script every loop? fork and abandon current thread?

TODO: update cell cache in intrepret notebooks.ipynb

TODO: git apply without whitespace, reset the rest?  separate index?  how do make git-scenario app?

TODO: re-import cells on Utilities/.modules folder change. run all "watcher" commands from a test below?  mocha.grep?





In [ ]:
var chokidar = require("chokidar");
var importer = require('../Core');
var testCells = importer.import('test cells');

// TODO: code analysis to combine blocks into modules?

var rateLimiter, done = true;
function testWatcher(files, tests) {
    files = typeof files === 'string' ? [files] : files;
    console.log('watching ' + files + ' - ' + path.resolve('.'))
    var watcher = chokidar.watch(files, {
        interval: 1000,
        atomic: 1000,
        awaitWriteFinish: true
    });
    watcher.on("change", function(event, path) {
        if(!done) {
            return;
        }
        console.log('running all tests');
        done = false;
        return testCells(tests).then(() => (done = true))
    });
    testCells(tests);
    
    var stdin = process.openStdin();
    stdin.addListener("data", function(d) {
        stdin.close();
    });
}
module.exports = testWatcher;
